In [ ]:
#coding:utf-8
import tensorflow as tf
tf.__version__
import sys
print(sys.version)

In [2]:
##计算图（Graph）:搭建神经网络的计算过程，只搭建，不运算
x=tf.constant([[1.0,2.0]])
w=tf.constant([[3.0],[4.0]])
y=tf.matmul(x,w)
print(y)

Tensor("MatMul:0", shape=(1, 1), dtype=float32)


In [3]:
##会话（Session）:执行计算图中的节点运算
with tf.Session() as sess:
    print(sess.run(y))

[[11.]]


- 两层简单神经网络（全连接）前向传播【喂一组数据】

In [4]:
##定义输入和参数
x= tf.constant([[0.7,0.5]])
w1= tf.Variable(tf.random_normal([2,3], stddev=1, seed=1))
w2= tf.Variable(tf.random_normal([3,1], stddev=1, seed=1))

##定义前向传播过程
a=tf.matmul(x,w1)
y=tf.matmul(a,w2)
print("a=",a)
print("y=",y)
##用会话计算结果
with tf.Session() as sess:
    init_op= tf.global_variables_initializer()
    sess.run(init_op) ##把网络的节点按照上面的参数进行设置初始参数
    print("y is:\n", sess.run(y))

a= Tensor("MatMul_1:0", shape=(1, 3), dtype=float32)
y= Tensor("MatMul_2:0", shape=(1, 1), dtype=float32)
y is:
 [[3.0904665]]


- 两层简单神经网络（全连接）前向传播【喂一组数据，用placeholder占位】

In [5]:
##定义输入和参数

##placeholder相当于输入的定义，但是现在未赋值
x=tf.placeholder(tf.float32,shape=(1,2))
w1= tf.Variable(tf.random_normal([2,3], stddev=1, seed=1))
w2= tf.Variable(tf.random_normal([3,1], stddev=1, seed=1))

##定义前向传播过程
a=tf.matmul(x,w1)
y=tf.matmul(a,w2)
print("a=",a)
print("y=",y)
##用会话计算结果
with tf.Session() as sess:
    init_op= tf.global_variables_initializer()
    sess.run(init_op) ##把网络的节点按照上面的参数进行设置初始参数
    print("y is:\n", sess.run(y, feed_dict={x:[[0.7,0.5]]}))

a= Tensor("MatMul_3:0", shape=(1, 3), dtype=float32)
y= Tensor("MatMul_4:0", shape=(1, 1), dtype=float32)
y is:
 [[3.0904665]]


- 两层简单神经网络（全连接）前向传播【喂多组数据，用placeholder占位】

In [6]:
##定义输入和参数

##placeholder相当于输入的定义，但是现在未赋值
x=tf.placeholder(tf.float32,shape=(None,2))
w1= tf.Variable(tf.random_normal([2,3], stddev=1, seed=1))
w2= tf.Variable(tf.random_normal([3,1], stddev=1, seed=1))

##定义前向传播过程
a=tf.matmul(x,w1)
y=tf.matmul(a,w2)
print("a=",a)
print("y=",y)
##用会话计算结果
with tf.Session() as sess:
    init_op= tf.global_variables_initializer()
    sess.run(init_op) ##把网络的节点按照上面的参数进行设置初始参数
    print("y is:\n", sess.run(y, feed_dict={x:[[0.7,0.5],[0.2,0.3],[0.3,0.4],[0.4,0.5]]}))
    print("w1:\n",sess.run(w1))
    print("w2=\n",sess.run(w2))

a= Tensor("MatMul_5:0", shape=(?, 3), dtype=float32)
y= Tensor("MatMul_6:0", shape=(?, 1), dtype=float32)
y is:
 [[3.0904665]
 [1.2236414]
 [1.7270732]
 [2.2305048]]
w1:
 [[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.0992484   0.5912243 ]]
w2=
 [[-0.8113182 ]
 [ 1.4845988 ]
 [ 0.06532937]]


反向传播，梯度优化，生成随机数据模拟二分类问题

In [7]:
import tensorflow as tf
import numpy as np
BATCH_SIZE=8
seed=23455

##基于seed产生随机数据集
rng=np.random.RandomState(seed)
##表示X数据有32组，两个特征，分别表示重量和体积
X=rng.rand(32,2)
##y标签，基于X的特点产生y.两个特征和小于1则y=0，大于1，y=0
Y= [[int(x0+x1 < 1) ] for x0,x1 in X]
print("X:\n",X)
print("Y:\n",Y)

##1.定义神经网络输入，参数和输出【前向传播过程】
x = tf.placeholder(tf.float32, shape=(None,2))
y_ = tf.placeholder(tf.float32, shape=(None,1))

w1=tf.Variable(tf.random_normal([2,3], stddev=1, seed=1))
w2=tf.Variable(tf.random_normal([3,1], stddev=1, seed=1))

a=tf.matmul(x,w1)
y=tf.matmul(a,w2)

##2.定义损失函数及传播方法
loss= tf.reduce_mean(tf.square(y-y_))
train_step = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

##3.生成会话，训练STEP
with tf.Session() as sess:
    init_op=tf.global_variables_initializer()
    sess.run(init_op)
    # 输出目前未经训练的参数权值
    print("w1:\n",sess.run(w1))
    print("w2:\n",sess.run(w2))
    
    # 训练模型
    STEPS = 3000 ## 每一次喂入一个BATCH_SIZE，喂STEPS轮。一共有32组数据，遍历完需要4轮。一共3000轮，遍历完数据的次数是3000/4
    for i in range(STEPS):
        start = (i*BATCH_SIZE) % 32
        end = start + BATCH_SIZE
        sess.run(train_step, feed_dict= {x: X[start:end], y_: Y[start:end]})
        if i % 500 == 0:
            total_loss= sess.run(loss, feed_dict= {x: X, y_: Y})
            print("After %d trian steps, loss on all data is %g" % (i, total_loss))
            
    # 输出训练后的参数取值
    print("w1:\n",sess.run(w1))
    print("w2:\n",sess.run(w2))

X:
 [[0.83494319 0.11482951]
 [0.66899751 0.46594987]
 [0.60181666 0.58838408]
 [0.31836656 0.20502072]
 [0.87043944 0.02679395]
 [0.41539811 0.43938369]
 [0.68635684 0.24833404]
 [0.97315228 0.68541849]
 [0.03081617 0.89479913]
 [0.24665715 0.28584862]
 [0.31375667 0.47718349]
 [0.56689254 0.77079148]
 [0.7321604  0.35828963]
 [0.15724842 0.94294584]
 [0.34933722 0.84634483]
 [0.50304053 0.81299619]
 [0.23869886 0.9895604 ]
 [0.4636501  0.32531094]
 [0.36510487 0.97365522]
 [0.73350238 0.83833013]
 [0.61810158 0.12580353]
 [0.59274817 0.18779828]
 [0.87150299 0.34679501]
 [0.25883219 0.50002932]
 [0.75690948 0.83429824]
 [0.29316649 0.05646578]
 [0.10409134 0.88235166]
 [0.06727785 0.57784761]
 [0.38492705 0.48384792]
 [0.69234428 0.19687348]
 [0.42783492 0.73416985]
 [0.09696069 0.04883936]]
Y:
 [[1], [0], [0], [1], [1], [1], [1], [0], [1], [1], [1], [0], [0], [0], [0], [0], [0], [1], [0], [0], [1], [1], [0], [1], [0], [1], [1], [1], [1], [1], [0], [1]]
w1:
 [[-0.8113182   1.4845988 